In [1]:
# Installing nsight-systems-2022.4.1 & nsight-compute-2022.1.1
!apt-get update -y && \
     DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
         apt-transport-https \
         ca-certificates \
         gnupg \
         wget && \
     rm -rf /var/lib/apt/lists/*
!wget -qO - https://developer.download.nvidia.com/devtools/repos/ubuntu2004/amd64/nvidia.pub | apt-key add - && \
     echo "deb https://developer.download.nvidia.com/devtools/repos/ubuntu2004/amd64/ /" >> /etc/apt/sources.list.d/nsight.list && \
     apt-get update -y && \
     DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
         nsight-systems-2022.1.1 nsight-compute-2022.1.1 && \
     rm -rf /var/lib/apt/lists/*

# setting environment variable path
import os
os.environ["PATH"] = "/usr/local/bin" + os.pathsep + os.getenv("PATH")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,227 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,287 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/

In [1]:
!git clone https://github.com/stprnvsh/nanoTabPFN.git

Cloning into 'nanoTabPFN'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 42 (delta 23), reused 28 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 24.00 KiB | 24.00 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [2]:
!cd nanoTabPFN/

In [4]:
!curl http://ml.informatik.uni-freiburg.de/research-artifacts/nanoTabPFN/300k_150x5_2.h5 --output 300k_150x5_2.h5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  989M  100  989M    0     0  15.3M      0  0:01:04  0:01:04 --:--:-- 14.5M


In [11]:
!python nanoTabPFN/generate_synthetic_data.py --scale 0.1 --rows 5000

Generating 30,000 samples -> 30k_5000x5_2.h5
Shape: X=(30000, 5000, 5), y=(30000, 5000)
Estimated size: 3.60 GB
  30,000/30,000 (100%)
Done! Saved to 30k_5000x5_2.h5
time: 27.7 s (started: 2026-01-03 15:33:25 +00:00)


In [4]:
!pip install schedulefree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.8 MB/s eta 0:00:00


In [5]:
!pip install ipython-autotime schedulefree
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.4 MB/s eta 0:00:00
time: 237 µs (started: 2026-01-03 15:29:53 +00:00)


In [43]:
!cd nanoTabPFN && python train.py --profile --batch-size=6 --steps=100

Device: cuda
Steps: 100
Batch size: 6
[W103 16:11:39.336264570 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event

=== TIMING ===
Total time: 68.75s
Steps/sec: 1.5
ms/step: 687.51

=== TOP CUDA OPERATIONS ===
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -----------

In [30]:
!cd nanoTabPFN && git stash && git pull

Saved working directory and index state WIP on main: 731be7b Add GDS (kvikio) dataloader for GPU Direct Storage
Already up to date.
time: 605 ms (started: 2026-01-03 15:55:23 +00:00)


In [42]:
!cd nanoTabPFN && python train_optimized.py --profile --no-compile --prefetch=8 --batch-size=6 --steps=100 --gds

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)
Device: cuda
GPU: NVIDIA A100-SXM4-80GB
TF32 enabled: True
torch.compile: False
Batch size: 6
VRAM prefetch: 8 batches
Data file: 30k_5000x5_2.h5
GDS (kvikio): True
Warming up (includes torch.compile time if enabled)...
GDSDataLoader: Using kvikio GPU Direct Storage
GDSDataLoader: 30,000 samples, 5 steps, batch_size=6
Warmup time: 5.25s

Profiling 100 steps...
GDSDataLoad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')